## Imports

In [1]:
from csv import reader
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import tree
from sklearn.externals.six import StringIO

## Read from CSV

In [2]:
fileValues = []

In [3]:
def readInValuesFromCSV():
    # open csv file and append to a list of dictionaries
    infile = open("NYPD_7_Major_Felony_Incidents.csv", 'r')
    first = True
    for line in reader( infile ):
#         print line
        if not first:
            OBJECTID,Identifier,Occurrence_Date,Day_of_Week,Occurrence_Month,Occurrence_Day,Occurrence_Year,Occurrence_Hour,CompStat_Month,CompStat_Day,CompStat_Year,Offense,Offense_Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location_1 = line
            
            values = {'OBJECTID' : OBJECTID,'Identifier':Identifier,'Occurrence Date':Occurrence_Date,'Day of Week':Day_of_Week,'Occurrence Month':Occurrence_Month,'Occurrence Day':Occurrence_Day,'Occurrence Year':Occurrence_Year,'Occurrence Hour':Occurrence_Hour,'CompStat Month':CompStat_Month,'CompStat Day':CompStat_Day,'CompStat Year':CompStat_Year,'Offense':Offense,'Offense Classification':Offense_Classification,'Sector':Sector,'Precinct':Precinct,'Borough':Borough,'Jurisdiction':Jurisdiction,'XCoordinate':XCoordinate,'YCoordinate':YCoordinate,'Location 1':Location_1 }
            
            fileValues.append(values)
            
        first = False
            

In [4]:
readInValuesFromCSV()

# Data Cleaning

In [9]:
# make sure that there are no empty values
crimeData = []
for crime in fileValues:
    if crime['Precinct'] != "" and crime['Offense'] != "" and crime['Day of Week'] != "" and crime['Occurrence Hour'] != "" and crime['Jurisdiction'] == "N.Y. POLICE DEPT":
        crimeData.append(crime)

## Data Value Encoding

In [12]:
# encode precints and offense into floats
offenseEncoder = preprocessing.LabelEncoder()
precinctEncoder = preprocessing.LabelEncoder()

In [13]:
offenseEncoder.fit([crime['Offense'] for crime in crimeData])
precinctEncoder.fit([crime['Precinct'] for crime in crimeData])

LabelEncoder()

## Split into training and test data

In [20]:
#split data into training data and test data
X_train, X_test, y_train, y_test = train_test_split(precinctEncoder.transform([x['Precinct'] for x in crimeData]),offenseEncoder.transform([x['Offense'] for x in crimeData]),test_size=0.10)

In [21]:
# init the decisionTreeClassifier and fit the training data
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train.reshape(-1,1),y_train.reshape(-1,1))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

## Prediction Score

In [22]:
dt.score(X_test.reshape(-1,1),y_test.reshape(-1,1))

0.38989436856623833

## Predict a crime

In [186]:
def predictCrimeInPrecinct(precinctNumber):
    return offenseEncoder.inverse_transform(
    dt.predict(
        [precinctEncoder.transform(
                [precinctNumber])]))